<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0_csv%E4%BF%9D%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/株式投資')

Mounted at /content/drive


In [ ]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=afe8ae0718b75c34f47752e5694d6e34ce7e58007b6e0e4ed7fa2ad5827e7cf0
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
!pip install yahoo_finance_api2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3714 sha256=a63ee4511d433b7defda81163dd692bcb075e5792ae9a01883cde4842ec63203
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [ ]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

# 東証上場の銘柄一覧を取得する関数（ダウンロードはしない）

In [ ]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    # サイトからダウンロードしないようにコメントアウト
    #url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    #r = requests.get(url)
    #with open('data_j.xls', 'wb') as output:
    #    output.write(r.content)

    df = pd.read_excel("./data/data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [ ]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [ ]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    result_list = []

    # データフレームを辞書のリストに変更
    df_to_dicts = df.to_dict(orient='records')

    # 各銘柄についてループ
    for row in df_to_dicts:
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        result_list.append({
            '銘柄名': row['銘柄名'],
            'コード': code,
            '業種': row['業種'],
            '出来高': volume,
            '株価': price,
        })

    # 辞書のリストをデータフレームに変換
    result_df = pd.DataFrame.from_dict(result_list)

    return result_df

# CSVに保存する関数

In [ ]:
import datetime as dt
import os

def save_df_as_csv(df, filename):
    """データフレームをCSVファイルとして保存する

    :param df: 対象のデータフレーム
    :param filename: 保存するcsvファイル名
    """
    # ディレクトリ名作成（YYYYMMDD_screening）
    now = dt.datetime.now().strftime("%Y%m%d")
    directory = f'{now}_screening'

    # もしディレクトリが存在しなければ、作成する
    if not os.path.exists(directory):
        os.makedirs(directory)

    # データフレームをCSVに保存する
    df.to_csv(f'{directory}/{filename}.csv', index=False)

# 東証プライムをフィルタリングしてCSVとして保存

In [ ]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4262,サンドラッグ,9989,小売業
4263,サックスバー ホールディングス,9990,小売業
4264,ジェコス,9991,卸売業
4267,グローセル,9995,卸売業


In [ ]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
import time
start_time = time.time()
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, max_price=5000)
print(f'経過時間: {time.time() - start_time}秒')
filtered_tse_prime

Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5128]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6924]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8096]の株価情報を取得できなかったため、スキップしました。
経過時間: 140.70387434959412秒


,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,858500,654.799988
1,ＩＮＰＥＸ,1605,鉱業,3421900,1630.500000
2,清水建設,1803,建設業,968100,892.000000
3,鹿島建設,1812,建設業,577100,2080.000000
4,積水ハウス,1928,建設業,1610100,2761.500000
...,...,...,...,...,...
178,レノバ,9519,電気・ガス業,511800,1570.000000
179,東京瓦斯,9531,電気・ガス業,505300,3225.000000
180,エイチ・アイ・エス,9603,サービス業,967900,2111.000000
181,エヌ・ティ・ティ・データ,9613,情報・通信業,917800,2134.500000


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,861400,655.000000
1,清水建設,1803,建設業,971400,892.299988
2,鹿島建設,1812,建設業,579000,2080.500000
3,積水ハウス,1928,建設業,1615200,2760.500000
4,ＬＩＦＵＬＬ,2120,サービス業,1473700,303.000000
...,...,...,...,...,...
86,北陸電力,9505,電気・ガス業,600300,764.200012
87,東北電力,9506,電気・ガス業,1088600,892.200012
88,九州電力,9508,電気・ガス業,519900,913.000000
89,北海道電力,9509,電気・ガス業,581500,606.599976


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,861400,655.000000
1,清水建設,1803,建設業,971400,892.299988
2,鹿島建設,1812,建設業,579000,2080.500000
3,ＬＩＦＵＬＬ,2120,サービス業,1473700,303.000000
4,円谷フィールズホールディングス,2767,卸売業,502700,2885.000000
...,...,...,...,...,...
76,北陸電力,9505,電気・ガス業,600300,764.200012
77,東北電力,9506,電気・ガス業,1088600,892.200012
78,九州電力,9508,電気・ガス業,519900,913.000000
79,北海道電力,9509,電気・ガス業,581500,606.599976


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_prime, '1_東証P_UPO')

In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,日本Ｍ＆Ａセンターホールディングス,2127,サービス業,3206800,1123.500000
1,エムスリー,2413,サービス業,1093800,3342.000000
2,インフォマート,2492,サービス業,781300,313.000000
3,伊藤園,2593,食料品,531800,4027.000000
4,グリー,3632,情報・通信業,853900,640.000000
5,ｇｕｍｉ,3903,情報・通信業,569400,687.000000
6,カナミックネットワーク,3939,情報・通信業,609700,485.000000
7,住友化学,4005,化学,4344200,437.899994
8,信越化学工業,4063,化学,2260600,4888.000000
9,ソースネクスト,4344,情報・通信業,732300,201.000000


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,伊藤園,2593,食料品,531800,4027.000000
1,グリー,3632,情報・通信業,853900,640.000000
2,ｇｕｍｉ,3903,情報・通信業,569400,687.000000
3,カナミックネットワーク,3939,情報・通信業,609700,485.000000
4,ソースネクスト,4344,情報・通信業,732300,201.000000
5,住友ファーマ,4506,医薬品,796100,664.299988
6,楽天グループ,4755,サービス業,21163100,514.400024
7,大阪チタニウムテクノロジーズ,5726,非鉄金属,839700,3015.000000
8,ジャパンディスプレイ,6740,電気機器,5675400,38.000000
9,シャープ,6753,電気機器,3237400,793.500000


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_prime, '2_東証P_DPO')

# 東証スタンダードをフィルタリングしてCSVとして保存

In [ ]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4257,タキヒヨー,9982,卸売業
4260,蔵王産業,9986,卸売業
4265,ヤマザワ,9993,小売業
4266,やまや,9994,小売業


In [ ]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, max_price=5000)
filtered_tse_standard

'timestamp'
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4616]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5271]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6647]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7509]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7515]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7677]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7957]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9063]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9778]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,235600,2555.0
1,住石ホールディングス,1514,鉱業,258500,370.0
2,中小企業ホールディングス,1757,建設業,1448400,47.0
3,アジアゲートホールディングス,1783,建設業,292300,57.0
4,燦キャピタルマネージメント,2134,サービス業,15330800,39.0
...,...,...,...,...,...
68,ビジョナリーホールディングス,9263,小売業,423400,128.0
69,ビート・ホールディングス・リミテッド,9399,情報・通信業,584240,6.0
70,ウィルソン・ラーニング ワールドワイド,9610,サービス業,204300,161.0
71,アゴーラ ホスピタリティー グループ,9704,サービス業,259300,26.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,258500,370.0
1,中小企業ホールディングス,1757,建設業,1448400,47.0
2,ＪＴＰ,2488,サービス業,277900,1089.0
3,イメージ ワン,2667,卸売業,310000,867.0
4,ｅｎｉｓｈ,3667,情報・通信業,4156000,625.0
5,パス,3840,情報・通信業,626400,115.0
6,ＰＫＳＨＡ Ｔｅｃｈｎｏｌｏｇｙ,3993,情報・通信業,813000,3335.0
7,細谷火工,4274,化学,284600,1675.0
8,ユークス,4334,情報・通信業,415800,2421.0
9,オハラ,5218,ガラス・土石製品,405400,1671.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,258500,370.0
1,ＪＴＰ,2488,サービス業,277900,1089.0
2,イメージ ワン,2667,卸売業,310000,867.0
3,ｅｎｉｓｈ,3667,情報・通信業,4156000,625.0
4,パス,3840,情報・通信業,626400,115.0
5,ＰＫＳＨＡ Ｔｅｃｈｎｏｌｏｇｙ,3993,情報・通信業,813000,3335.0
6,細谷火工,4274,化学,284600,1675.0
7,ユークス,4334,情報・通信業,415800,2421.0
8,オハラ,5218,ガラス・土石製品,405400,1671.0
9,ダントーホールディングス,5337,ガラス・土石製品,211100,740.0


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_standard, '3_東証S_UPO')

In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,235600,2555.0
1,アジアゲートホールディングス,1783,建設業,292300,57.0
2,クルーズ,2138,情報・通信業,223600,951.0
3,クシム,2345,情報・通信業,549900,332.0
4,ピクセルカンパニーズ,2743,卸売業,467900,45.0
5,ブロードバンドタワー,3776,情報・通信業,2420100,158.0
6,リミックスポイント,3825,情報・通信業,386400,241.0
7,千代田化工建設,6366,建設業,755600,371.0
8,ウインテスト,6721,電気機器,255300,115.0
9,ピクセラ,6731,電気機器,1915800,2.0


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,235600,2555.0
1,クシム,2345,情報・通信業,549900,332.0
2,ピクセルカンパニーズ,2743,卸売業,467900,45.0
3,リミックスポイント,3825,情報・通信業,386400,241.0
4,ウインテスト,6721,電気機器,255300,115.0
5,ピクセラ,6731,電気機器,1915800,2.0
6,大黒屋ホールディングス,6993,小売業,929300,55.0
7,プロルート丸光,8256,卸売業,356600,47.0
8,第一商品,8746,証券、商品先物取引業,223100,106.0
9,ＧＦＡ,8783,その他金融業,234800,80.0


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_standard, '4_東証S_DPO')

# 東証グロースをフィルタリングしてCSVとして保存

In [ ]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
55,ルーデン・ホールディングス,1400,建設業
56,エムビーエス,1401,建設業
65,Ｌｉｂ Ｗｏｒｋ,1431,建設業
70,フィット,1436,建設業
75,ニッソウ,1444,建設業
...,...,...,...
4079,プログリット,9560,サービス業
4080,グラッドキューブ,9561,サービス業
4081,ビジネスコーチ,9562,サービス業
4082,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [ ]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, max_price=5000)
filtered_tse_grows

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,762000,1054.0
1,ＦＲＯＮＴＥＯ,2158,サービス業,833400,759.0
2,ジーエヌアイグループ,2160,医薬品,6153000,1628.0
3,メディネット,2370,サービス業,2429000,71.0
4,ウェッジホールディングス,2388,その他金融業,299700,111.0
...,...,...,...,...,...
98,マイクロ波化学,9227,サービス業,1031100,1863.0
99,サンウェルズ,9229,サービス業,293600,3730.0
100,ＧＥＮＯＶＡ,9341,サービス業,433400,2419.0
101,ｉｓｐａｃｅ,9348,サービス業,576500,1716.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,海帆,3133,小売業,4221500,638.0
1,トラストホールディングス,3286,不動産業,210900,375.0
2,モブキャストホールディングス,3664,情報・通信業,433800,84.0
3,ＦＦＲＩセキュリティ,3692,情報・通信業,431300,1708.0
4,アドバンスト・メディア,3773,情報・通信業,540800,1990.0
5,Ａｉｍｉｎｇ,3911,情報・通信業,1119200,464.0
6,シェアリングテクノロジー,3989,情報・通信業,649400,778.0
7,フィーチャ,4052,情報・通信業,470400,1394.0
8,ティアンドエス,4055,情報・通信業,389700,3240.0
9,プレイド,4165,情報・通信業,2440100,919.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,海帆,3133,小売業,4221500,638.0
1,トラストホールディングス,3286,不動産業,210900,375.0
2,モブキャストホールディングス,3664,情報・通信業,433800,84.0
3,ＦＦＲＩセキュリティ,3692,情報・通信業,431300,1708.0
4,シェアリングテクノロジー,3989,情報・通信業,649400,778.0
5,フィーチャ,4052,情報・通信業,470400,1394.0
6,ティアンドエス,4055,情報・通信業,389700,3240.0
7,プレイド,4165,情報・通信業,2440100,919.0
8,ＥＮＥＣＨＡＮＧＥ,4169,情報・通信業,687600,1485.0
9,セキュア,4264,情報・通信業,294400,3440.0


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_grows, '5_東証G_UPO')

In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,762000,1054.0
1,メディネット,2370,サービス業,2429000,71.0
2,アスカネット,2438,サービス業,257900,853.0
3,サンワカンパニー,3187,小売業,443400,657.0
4,ディー・ディー・エス,3782,情報・通信業,374100,50.0
5,グローバルウェイ,3936,情報・通信業,346000,227.0
6,Ｋａｉｚｅｎ Ｐｌａｔｆｏｒｍ,4170,情報・通信業,221300,357.0
7,ココナラ,4176,情報・通信業,356500,403.0
8,アクリート,4395,情報・通信業,646900,1984.0
9,ミンカブ・ジ・インフォノイド,4436,情報・通信業,741300,1883.0


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,アスカネット,2438,サービス業,257900,853.0
1,リボミック,4591,医薬品,229300,164.0
2,プレミアアンチエイジング,4934,化学,229800,1203.0
3,トレンダーズ,6069,サービス業,329900,1199.0
4,ＮｅｘＴｏｎｅ,7094,サービス業,220000,2196.0


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_grows, '6_東証G_DPO')

# 東証スタンダード・プライム選抜をCSVとして保存

In [ ]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4616]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5271]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6647]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7509]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7515]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7677]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7957]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9063]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9778]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,中小企業ホールディングス,1757,建設業,1448400,47.0
1,燦キャピタルマネージメント,2134,サービス業,15330800,39.0
2,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,1984500,67.0
3,クシム,2345,情報・通信業,549900,332.0
4,出前館,2484,情報・通信業,714700,437.0
5,レカム,3323,卸売業,1233400,91.0
6,ｅｎｉｓｈ,3667,情報・通信業,4156000,625.0
7,ブロードバンドタワー,3776,情報・通信業,2420100,158.0
8,パス,3840,情報・通信業,626400,115.0
9,ＰＫＳＨＡ Ｔｅｃｈｎｏｌｏｇｙ,3993,情報・通信業,813000,3335.0


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2, '7_東証S_選抜')

In [ ]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_grows, min_volume=500000, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,762000,1054.0
1,ＦＲＯＮＴＥＯ,2158,サービス業,833400,759.0
2,ジーエヌアイグループ,2160,医薬品,6153000,1628.0
3,メディネット,2370,サービス業,2429000,71.0
4,海帆,3133,小売業,4221500,638.0
5,アドバンスト・メディア,3773,情報・通信業,540800,1990.0
6,ＦＨＴホールディングス,3777,情報・通信業,725500,36.0
7,Ａｉｍｉｎｇ,3911,情報・通信業,1119200,464.0
8,シェアリングテクノロジー,3989,情報・通信業,649400,778.0
9,プレイド,4165,情報・通信業,2440100,919.0


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2, '8_東証G_選抜')